# Query Gaia catalog

In [6]:
import numpy as np
from seismolab.gaia import query_gaia

targets = np.array([3107142829863485952])

In [7]:
res = query_gaia(targets,gaiaDR=3)

Using Combined19 map from mwdust
Querying Gaia DR3 and BJ catalogs...
Calculating distances, absolute magnitudes...


  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
res

Source,ra,dec,plx,sig_plx,gamag,sig_gamag,bpmag,sig_bpmag,rpmag,sig_rpmag,vmag,sig_vmag,jmag,sig_jmag,hmag,sig_hmag,kmag,sig_kmag,dis,disep,disem,lon_deg,lat_deg,ag,agep,agem,MG,MGep,MGem,mG,mGep,mGem,aBP,aBPep,aBPem,MBP,MBPep,MBPem,mBP,mBPep,mBPem,aRP,aRPep,aRPem,MRP,MRPep,MRPem,mRP,mRPep,mRPem,aV,aVep,aVem,MV,MVep,MVem,mV,mVep,mVem,aJ,aJep,aJem,MJ,MJep,MJem,mJ,mJep,mJem,aH,aHep,aHem,MH,MHep,MHem,mH,mHep,mHem,aK,aKep,aKem,MK,MKep,MKem,mK,mKep,mKem
,deg,deg,mas,mas,mag,,mag,,mag,,mag,,mag,,mag,,mag,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
str21,float64,float64,float64,float32,float32,float64,float32,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
3107142829863485952,101.76822041702867,-1.2075151625364267,0.432227248321794,0.030997224,12.621653,0.0006344411880043353,12.802992,0.0020885144822050884,12.291391,0.0017593474516210648,12.72,0.020,11.903,0.023,11.838,0.023,11.8,0.024,2190.859619140625,197.47900390625,151.5408935546875,213.5092718420073,-1.5216790901099373,1.0972506348348021,0.2813463101744822,0.01746006095976571,-0.17867081801656326,0.17374130314264136,0.46938729127948164,11.524401968314612,0.018094502147770086,0.2819807513624877,1.3107321146267983,0.3360851498919084,0.020857096725901858,-0.21081303389254558,0.1785924122029794,0.5255802042911046,11.492259752438631,0.022945611208106342,0.33817366437411245,0.7576583159506094,0.19427135861061418,0.012056279543791626,-0.16933972960112342,0.1694624279902861,0.383437245979227,11.533733056730055,0.013815626995413055,0.1960307060622366,1.1924453025900386,0.305755198782921,0.01897485133607879,-0.17551782189240583,0.19462211009462393,0.5131621964635862,11.527554964438771,0.03897530909974911,0.325755656546594,0.3091529886603355,0.07926999527513978,0.0049194139028371775,-0.10922589706182606,0.1835659783864827,0.2896762986808987,11.593846889269352,0.027919177391606098,0.10226975876391009,0.18290999329090118,0.04689999720454216,0.00291056530898276,-0.047982482075692445,0.18155712979262617,0.2573063006103027,11.655090304255486,0.025910328797751347,0.06989976069331227,0.11801399567127227,0.030259998196363436,0.001877904184431095,-0.021086591267586385,0.1815248730259853,0.2416667059600357,11.68198619506359,0.025878072031112254,0.054260166043041735


In [9]:
res2 = query_gaia(targets,gaiaDR=2)

Using Combined19 map from mwdust
Querying Gaia DR2 catalog...
Calculating distances, absolute magnitudes...


  0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
res2

Source,ra,dec,plx,sig_plx,gamag,sig_gamag,bpmag,sig_bpmag,rpmag,sig_rpmag,vmag,sig_vmag,jmag,sig_jmag,hmag,sig_hmag,kmag,sig_kmag,dis,disep,disem,lon_deg,lat_deg,ag,agep,agem,MG,MGep,MGem,mG,mGep,mGem,aBP,aBPep,aBPem,MBP,MBPep,MBPem,mBP,mBPep,mBPem,aRP,aRPep,aRPem,MRP,MRPep,MRPem,mRP,mRPep,mRPem,aV,aVep,aVem,MV,MVep,MVem,mV,mVep,mVem,aJ,aJep,aJem,MJ,MJep,MJem,mJ,mJep,mJem,aH,aHep,aHem,MH,MHep,MHem,mH,mHep,mHem,aK,aKep,aKem,MK,MKep,MKem,mK,mKep,mKem
,deg,deg,mas,mas,mag,,mag,,mag,,mag,,mag,,mag,,mag,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
str21,float64,float64,float64,float64,float32,float64,float32,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
3107142829863485952,101.76822078135328,-1.207515192087555,0.2784408581785764,0.07150647466161306,12.631575,0.0009382113738554969,12.801229,0.0038396854144956893,12.298964,0.002020975902707495,12.72,0.020,11.903,0.023,11.838,0.023,11.8,0.024,3832.632796327963,1205.1134011340114,781.2820460204603,213.50927203436055,-1.521678779347881,1.7696987325866222,0.4122728676707361,0.2407382729227452,-2.0556418277891266,0.7348429400105714,1.005430029894729,10.861819577586186,0.2396506499506046,0.4117372038317537,2.1140119571358085,0.49248482569953156,0.2875763982466415,-2.2304442769199397,0.7812866080966312,1.0848303808647772,10.687120093832506,0.28631905727537976,0.49173506812071466,1.221987865765398,0.2846769522977459,0.1662312589821069,-1.8407000304353147,0.660724716880748,0.8775676772207697,11.07694807645693,0.16551958658712707,0.28428683795836207,1.923233282968368,0.44804061062430733,0.2616241117450673,-2.1210607288462935,0.7566343652693712,1.0398941209468324,10.796809821669349,0.2612950590289671,0.4473505455900977,0.4986168472711183,0.11615886574826562,0.06782858373119044,-1.5138191409378114,0.5639251848339407,0.7078604952534233,11.404163447704212,0.07020757389433818,0.11018300235435596,0.2950060569826599,0.0687252529783482,0.04013069984852824,-1.3751904042616552,0.5365573045018889,0.6604500211164646,11.541188201068914,0.0460037103824984,0.06204263123418663,0.19033866277815115,0.044341709064495,0.025892430222099444,-1.3084410644785818,0.5224782172061465,0.636526800950777,11.607118544052254,0.03606880692052172,0.04095751794617364
